<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/TAPT_base_pretrain_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This trains a roberta-base model on 100 epochs of TAPT helpfulness

At epoch 20 was eval score of 1.472466

# Settings
* Set batch size to 21, gradient_accumulation_steps=2, ~6 reviews per row = 252 batch size (paper has 256)
* LR 0.0001
* 100 epochs
* torch_compile=True (Because no adapter)
* early_stopping_patience=10
* no warmup







In [2]:
import torch

%load_ext autoreload
%autoreload 2


%env CUDA_VISIBLE_DEVICES=4, 5, 6, 7
#%env CUDA_DEVICE_ORDER="PCI_BUS_ID"
import os
# Set TOKENIZERS_PARALLELISM to false

os.environ["TOKENIZERS_PARALLELISM"] = "false"

print(torch.cuda.device_count())  # Should match the number of GPUs you specified
print(torch.cuda.get_device_name(0))

env: CUDA_VISIBLE_DEVICES=4, 5, 6, 7
4
NVIDIA RTX A4000


# Setup

In [3]:
from time import time
start_time = time()

# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

#!pip install datasets
#!pip install transformers[torch]
# !pip install adapters
# !pip install scikit-learn

#!pip install pynvml

In [4]:
from sklearn.metrics import accuracy_score, f1_score
import json

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
def load_json(path):
    with open(path) as f:
        data = [json.loads(line) for line in f]
    return data

def save_json(data, filename):
    with open(filename, 'w') as file:
        for r in data:
            json.dump(r, file)
            file.write('\n')

# Save data locally

In [101]:
create_dir('datasets')
create_dir('datasets/classifier')
create_dir('datasets/classifier/amazon')

In [14]:
# !curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/train.jsonl --output-dir 'datasets/classifier/amazon'
# !curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/dev.jsonl --output-dir 'datasets/classifier/amazon'
# !curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/test.jsonl --output-dir 'datasets/classifier/amazon'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  100M  100  100M    0     0  29.2M      0  0:00:03  0:00:03 --:--:-- 29.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4523k  100 4523k    0     0  5430k      0 --:--:-- --:--:-- --:--:-- 5430k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.7M  100 21.7M    0     0  1972k      0  0:00:11  0:00:11 --:--:-- 2816k


In [3]:
train_json_h = load_json("datasets/classifier/amazon/train.jsonl")
dev_json_h = load_json("datasets/classifier/amazon/dev.jsonl")
test_json_h = load_json("datasets/classifier/amazon/test.jsonl")
print("Helfulness ds stat")
print("Train ds: {0}".format(len(train_json_h)))
print("Dev ds: {0}".format(len(dev_json_h)))
print("Test ds: {0}".format(len(test_json_h)))

Helfulness ds stat
Train ds: 115251
Dev ds: 5000
Test ds: 25000


In [33]:
train_json_new = []
dev_json_new = []

In [37]:
def reformat_json(json):
    json_new = []
    for r in json:
        text = r['text']
        split_text = text.splitlines()
        for s in split_text:
            if len(s) > 0 and not s.isspace():
                json_new.append({'text':s})
    return json_new

In [38]:
train_json_new = reformat_json(train_json_h)
print(len(train_json_new))

230176


In [40]:
dev_json_new = reformat_json(dev_json_h)
print(len(dev_json_new))

9988


In [8]:
# save_json(train_json_new, 'train_new.jsonl')
# save_json(dev_json_new, 'dev_new.jsonl')

In [4]:
# ## Here I tried to reproduce what it seems they did in repo - split reviews line-by-line.
# ## However it gave me 2x of rows, and I ended-up going with initial review text without splitting.

# from datasets import load_dataset
# dataset = load_dataset("json",
#                        data_files={"train":"datasets/classifier/amazon/train_new.jsonl",
#                                    "dev":"datasets/classifier/amazon/dev_new.jsonl"})

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-60c89eecf49b1ac7/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from datasets import load_dataset
dataset = load_dataset("json",
                       data_files={"train":"datasets/classifier/amazon/train.jsonl",
                                   "dev":"datasets/classifier/amazon/dev.jsonl"})

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-ea5ce09c80d3e0ed/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 115251
    })
    dev: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 5000
    })
})

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [8]:
dataset.column_names

{'train': ['id', 'text', 'label'], 'dev': ['id', 'text', 'label']}

In [9]:
def tokenize_function(examples):
    # not sure it is needed, but they used add_special_tokens=True in the repo
    return tokenizer(examples['text'], add_special_tokens=True, max_length=512)

dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/115251 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 115251
    })
    dev: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [12]:
dataset.push_to_hub("ltuzova/amazon_helpfulness_TAPT_pretraining_dataset_no_condence")

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/116 [00:00<?, ?ba/s]

Pushing split dev to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

In [10]:
from transformers import AutoConfig, AutoModelForMaskedLM
pretrain_model = AutoModelForMaskedLM.from_pretrained("roberta-base")

In [11]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [12]:
from transformers import TrainingArguments

# Just 10 epochs here, as it takes too long to run 100. 

pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_base_pretraining_no_condencing",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=32, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=10, #100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    torch_compile=False,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="wandb",
)

In [13]:
from transformers import EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [14]:
from transformers import Trainer

pretrain_trainer = Trainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/home/jupyter/dental-notebooks/ltuzova/gatech_dl/final/AdaptOrDie/tapt_helpfulness_base_pretraining_no_condencing is already a clone of https://huggingface.co/ltuzova/tapt_helpfulness_base_pretraining_no_condencing. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
wandb: Currently logged in as: lyudmila. Use `wandb login --relogin` to force relogin


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,1.637100,1.452468
1,1.510700,1.410233
2,1.487100,1.384545


/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [16]:
import pandas as pd

pd.DataFrame(pretrain_trainer.state.log_history)

,loss,learning_rate,epoch,step,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,1.6371,0.00009,1.00,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,1.00,112,1.452468,46.8888,106.635,1.685,NaN,NaN,NaN,NaN,NaN
2,1.5107,0.00008,2.00,225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2.00,225,1.410233,46.4018,107.754,1.703,NaN,NaN,NaN,NaN,NaN
4,1.4871,0.00007,2.99,337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.99,337,1.384545,46.3772,107.812,1.703,NaN,NaN,NaN,NaN,NaN
6,1.4450,0.00006,4.00,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,4.00,450,1.377782,46.3941,107.772,1.703,NaN,NaN,NaN,NaN,NaN
8,1.4412,0.00005,4.99,562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,4.99,562,1.348311,46.5036,107.519,1.699,NaN,NaN,NaN,NaN,NaN


In [17]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

/home/jupyter/dental-notebooks/ltuzova/.ltuzova_env/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


After pretraining loss: 1.3379
{'eval_loss': 1.337902545928955, 'eval_runtime': 45.4134, 'eval_samples_per_second': 110.1, 'eval_steps_per_second': 1.74, 'epoch': 9.95}


In [18]:
pretrain_trainer.push_to_hub("tapt_helpfulness_base_pretraining_no_condencing_10_epochs")

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/476M [00:00<?, ?B/s]

To https://huggingface.co/ltuzova/tapt_helpfulness_base_pretraining_no_condencing
   edf6f9f..e3126bb  main -> main

To https://huggingface.co/ltuzova/tapt_helpfulness_base_pretraining_no_condencing
   e3126bb..ab191c8  main -> main



'https://huggingface.co/ltuzova/tapt_helpfulness_base_pretraining_no_condencing/commit/e3126bb3d77a8158be8d7d24f890a0b49c3b3bb9'

In [19]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

Total Time: 543 minutes OR 32580 seconds


# Classification

In [19]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
